In [31]:
# Import required libraries:

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn import metrics as met

In [108]:
categories = {1: 'Algida',
             2: 'CCI_TR_Mini_Pilot_2019',
             3: 'Mauritus_Pilot',
             4: 'PG_TR_Pilot_2019_laundry',
             5: 'Ulker_Project_2018'}

category_number = 5 # Select the category to work on

df = pd.read_csv('C:/Users/Mecha/Desktop/Graduate_Courses/EE58J_DataMining/HW3/{}.csv'.format(categories[category_number]), index_col = 0, header = None)
X = df.values
index_list = list(df.index)
index_list = [im.split('/')[0] for im in index_list]
y = np.asarray(index_list)
classnames, y = np.unique(y, return_inverse=True)
print(X.shape, y.shape, classnames.shape)

(2065, 1536) (2065,) (20,)


In [109]:
# Split the data into test and training data. By choosing stratify=y, each class will be split approximately by 20-80 ratio. 
X_tr, X_test, y_tr, y_test = model_selection.train_test_split(X, y,
                                                random_state = 2,
                                                stratify=y,
                                                test_size=0.20)
print(X_tr.shape, y_tr.shape)

(1652, 1536) (1652,)


In [120]:
number_of_folds = 10 # Choose the number of folds for Cross Validation
kf = KFold(n_splits=number_of_folds, random_state=1, shuffle=True)

#C_values = [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2]
C_values = [0.02]
mean_accuracies = np.zeros( (len(C_values), 1) )

for i,c in enumerate(C_values):
    print(c)
    mean_accuracy = 0
    for train_index, cv_index in kf.split(X_tr):
        X_train, X_cv = X_tr[train_index], X_tr[cv_index]
        y_train, y_cv = y_tr[train_index], y_tr[cv_index]
    
        model = SVC(C=c, kernel='linear')
        model.fit(X_train, y_train)
        fold_accuracy = model.score(X_cv, y_cv)
        mean_accuracy += fold_accuracy / number_of_folds
    mean_accuracies[i] = mean_accuracy

0.02


In [121]:
print(mean_accuracies*100)

[[93.40270172]]


In [122]:
model = SVC(C=0.02, kernel='linear') # Choose the regularization parameter found by above search
model.fit(X_tr, y_tr)
test_accuracy = model.score(X_test, y_test)
tr_accuracy = model.score(X_tr, y_tr)

print(test_accuracy*100, tr_accuracy*100)

89.34624697336562 99.21307506053269


In [123]:
# Print the confusion matrix:
conf = met.confusion_matrix(y_test, model.predict(X_test))
print(pd.DataFrame(conf).iloc[:,:])

     0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  \
0   19   1   1   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
1    0  17   2   0   0   0   0   0   0   1   0   0   0   0   0   0   0   0   
2    0   1  21   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   
3    0   0   0  18   0   0   0   0   1   0   1   0   0   0   0   0   0   0   
4    0   0   0   0  20   0   0   0   0   0   0   0   0   0   0   0   0   0   
5    0   0   0   0   0  21   0   0   0   0   0   0   0   0   0   0   0   0   
6    0   0   0   0   0   1  20   0   0   0   0   0   0   0   0   0   0   0   
7    0   0   0   0   0   0   0  21   0   0   0   0   0   0   0   0   0   0   
8    0   0   0   0   0   0   0   1  20   0   0   0   0   0   0   0   0   0   
9    0   0   0   0   0   0   0   0   1  19   0   0   0   0   0   0   0   0   
10   0   0   0   0   0   0   0   0   0   0  20   0   0   0   0   1   0   0   
11   0   0   0   0   0   0   0   0   0   0   0  11   9   1   0  

In [124]:
classnames

array(['6753', '6764', '6770', '6783', '6784', '6799', '6800', '6820',
       '6822', '6828', '6831', '6866', '6867', '7488', '7489', '9554',
       '9555', '9556', '9557', '9578'], dtype='<U4')